In [73]:
from casPYan import Node, Edge, connect, step, apply_spike
import casPYan as cap
import tabulate
import json

from IPython.core.debugger import Pdb

In [74]:
def network_table(nodes):
    # Show network output/state
    counts = cap.fires(nodes)
    vectors = cap.vectors(nodes)
    lastfires = cap.lastfires(nodes)
    charges = cap.charges(nodes)
    # histories = [node.history for node in nodes]

    data = zip(
        range(len(counts)),
        charges,
        counts,
        lastfires,
        vectors,
        # histories,
    )
    data = [[
        'id',
        'charge',
        'fires',
        't_lastfire',
        't_fires',
        # 'histories',
    ]] + list(data)
    return tabulate.tabulate(data, tablefmt='html')

In [75]:
nodes = []  # create nodes object
inputs = i0, i1 = Node(1), Node(1)  # create true input nodes
outputs = out = [Node(1) for i in range(4)]  # create true output nodes

nodes += inputs  # add inputs and outputs to the list of all nodes
nodes += outputs

In [76]:
# apply_spike(in0, 1.0, 0)
def apply_spike_const(node, a, n):
    node.intake += [(a, delay) for delay in range(n)]

def read_jsonfile(path):
    with open(path, 'r') as f:
        return json.loads(f.read())

nets = [cap.network_from_json(read_jsonfile(f"networks/b2_{i}.json")) for i in range(10 + 1)]


In [77]:
def foldin_subnet(subnet, output, invert=False):
    global nodes
    subnodes, subinputs, suboutputs = subnet
    nodes += list(subnodes.values())
    connect(inputs[1 if invert else 0], subinputs[0], weight=255)
    connect(inputs[0 if invert else 1], subinputs[1], weight=255)
    connect(suboutputs[0], output, weight=255)


# see stuff
foldin_subnet(nets[2], out[1])  # +v
foldin_subnet(nets[2], out[3])  # +w
# don't see stuff
foldin_subnet(nets[2], out[1], invert=1)
foldin_subnet(nets[2], out[3], invert=1)

In [78]:
# cap.step(nodes)
# inputs[1].callback_prestep_fire = lambda node: Pdb().set_trace()

In [79]:
apply_spike_const(inputs[1], 255, 10)

In [80]:
cap.run(nodes, 5)
cap.run(nodes, 10)
# cap.run(nodes, 0)

In [81]:
network_table(nodes)

id,charge,fires,t_lastfire,t_fires
0,0,0,-1.0,[]
1,0,6,5,"[0, 1, 2, 3, 4, 5]"
2,0,0,-1.0,[]
3,0,3,8,"[2, 5, 8]"
4,0,0,-1.0,[]
5,0,3,8,"[2, 5, 8]"
6,0,7,24,"[0, 4, 8, 12, 16, 20, 24]"
7,0,7,24,"[0, 4, 8, 12, 16, 20, 24]"
8,45.0,3,28,"[4, 16, 28]"
9,0,0,-1.0,[]


In [82]:


# nodes, inputs, outputs = cap.network_from_json(j)
nodes

[Node at 7f27eb27cce0 w/ Threshold: 1, Delay: None, Leak: None, children: ['4bc0', '5b20'],
 Node at 7f27eb27d370 w/ Threshold: 1, Delay: None, Leak: None, children: ['5b20', '4bc0'],
 Node at 7f27eb27d220 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f27eb27cb30 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f27eb2740e0 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f27eb276720 w/ Threshold: 1, Delay: None, Leak: None, children: [],
 Node at 7f27eb274bc0 w/ Threshold: 45.0, Delay: 0.0, Leak: 1.0, children: ['5b20', '4bc0'],
 Node at 7f27eb275b20 w/ Threshold: 110.0, Delay: 0.0, Leak: 2.0, children: ['4bc0', '4d40'],
 Node at 7f27eb274d40 w/ Threshold: 109.0, Delay: 0.0, Leak: None, children: ['cb30', '6720'],
 Node at 7f27eb276ab0 w/ Threshold: 75.0, Delay: 0.0, Leak: 1.0, children: [],
 Node at 7f27eb2766c0 w/ Threshold: 70.0, Delay: 0.0, Leak: 2.0, children: [],
 Node at 7f27eb2753d0 w/ Threshold: 31.0, Delay: 0.0, Leak: 3.0,

In [83]:
d = cap.to_tennlab(nodes, inputs, outputs)
d['Associated_Data']['application'].update({'proc_ticks': 10})
j = json.dumps(d)
with open('networks/combonet_vv_1.json', 'w') as f:
    f.write(j)
d

{'Associated_Data': {'application': {'proc_ticks': 10},
  'label': None,
  'processor': {'Leak_Enable': True,
   'Max_Leak': 4,
   'Min_Leak': -1,
   'Max_Weight': 127,
   'Min_Weight': -127,
   'Max_Threshold': 127,
   'Min_Threshold': 0,
   'Max_Synapse_Delay': 255,
   'Min_Synapse_Delay': 0,
   'Max_Axon_Delay': 0,
   'Min_Axon_Delay': 0}},
 'Properties': {'network_properties': [],
  'node_properties': [{'index': 0,
    'name': 'Threshold',
    'max_value': 127.0,
    'min_value': 0.0,
    'size': 1,
    'type': 73},
   {'index': 1,
    'name': 'Leak',
    'max_value': 4.0,
    'min_value': -1.0,
    'size': 1,
    'type': 73},
   {'index': 2,
    'name': 'Delay',
    'max_value': 0.0,
    'min_value': 0.0,
    'size': 1,
    'type': 73}],
  'edge_properties': [{'index': 0,
    'name': 'Weight',
    'max_value': 127.0,
    'min_value': -127.0,
    'size': 1,
    'type': 73},
   {'index': 1,
    'name': 'Delay',
    'max_value': 300.0,
    'min_value': 0.0,
    'size': 1,
    'type':

In [84]:
nodes, inputs, outputs = cap.network_from_json(read_jsonfile(f"networks/combonet_vv_1.json"))
nodes = nodes.values()

In [85]:
apply_spike_const(inputs[0], 255, 10)

In [86]:
cap.run(nodes, 5)
cap.run(nodes, 10)
# cap.run(nodes, 0)

In [87]:
network_table(nodes)

id,charge,fires,t_lastfire,t_fires
0,0,6,5,"[0, 1, 2, 3, 4, 5]"
1,0,0,-1.0,[]
2,0,0,-1.0,[]
3,0,2,8,"[5, 8]"
4,0,0,-1.0,[]
5,0,2,8,"[5, 8]"
6,0.0,7,6,"[0, 1, 2, 3, 4, 5, 6]"
7,0.0,7,6,"[0, 1, 2, 3, 4, 5, 6]"
8,135.0,2,7,"[4, 7]"
9,0,0,-1.0,[]
